In [12]:
# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI

from neo4j import GraphDatabase

import csv
import pandas as pd
import time
import os

# Common data processing
import json
import textwrap

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [13]:
# Load from environment
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'

In [14]:
# load the data
df = pd.read_csv('/mnt/code/data/netflix_titles.csv')
df["date_added"] = pd.to_datetime(df['date_added'])
df['year'] = df['date_added'].dt.year
df['month'] = df['date_added'].dt.month
df['day'] = df['date_added'].dt.day

df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year,month,day
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,2019.0,9.0,9.0
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,2016-09-09,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,2016.0,9.0,9.0
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,2018-09-08,2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob...",2018.0,9.0,8.0
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,2018-09-08,2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...,2018.0,9.0,8.0
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,2017-09-08,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,2017.0,9.0,8.0


In [16]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [ ]:
# DELETE ALL NODES AND RELATIONSHIPS
request_delete ="""
MATCH (n) DETACH DELETE n;
"""
kg.query(request_delete)

In [ ]:
csv_file_path = "/mnt/code/data/netflix_titles.csv"

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Function to insert data from CSV to Neo4j
def insert_data_from_csv(driver, csv_file_path):
    with driver.session() as session:
        with open(csv_file_path, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                session.run("""
                    CREATE (m:Movie {
                        id: $show_id,
                        type: $type,
                        title: $title,
                        director: $director,
                        cast: $cast,
                        country: $country,
                        date_added: $date_added,
                        release_year: $release_year,
                        rating: $rating,
                        duration: $duration,
                        listed_in: $listed_in,
                        description: $description
                    })
                """, parameters=row)

# Execute the function
insert_data_from_csv(driver, csv_file_path)

In [ ]:
# Cypher queries to setup the nodes and relationships
create_acted_in_rel = """
MATCH (m:Movie)
WHERE m.cast IS NOT NULL
WITH m
UNWIND split(m.cast, ',') AS actor
MERGE (p:Person {name: trim(actor)})
MERGE (p)-[r:ACTED_IN]->(m);
"""

create_category ="""    
MATCH (m:Movie)
WHERE m.listed_in IS NOT NULL
WITH m
UNWIND split(m.listed_in, ',') AS category
MERGE (c:Category {name: trim(category)})
MERGE (m)-[r:IN_CATEGORY]->(c);
"""    

# CREATE TYPE
create_type = """ 
MATCH (m:Movie)
WHERE m.type IS NOT NULL
WITH m
MERGE (t:Type {type: m.type})
MERGE (m)-[r:TYPED_AS]->(t);
"""

# CREATE DIRECTOR
create_director = """ 
MATCH (m:Movie)
WHERE m.director IS NOT NULL
WITH m
MERGE (d:Person {name: m.director})
MERGE (d)-[r:DIRECTED]->(m);
"""

# CREATE COUNTRIES
create_countries = """ 
MATCH (m:Movie)
WHERE m.country IS NOT NULL
MERGE (c:Country {name: trim(m.country)})
MERGE (m)-[:WHERE]->(c);
"""


# DELETE UNUSED PROPERTY; you can skip this if you want
del_unused = """
MATCH(m:Movie)
SET m.country = null, m.category = null,m.type = null , m.director = null , m.cast =null
"""

# BUILD THE WORK_WITH RELATIONSHIP
create_work_with_rel = """
MATCH (p:Person)-[rel:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(p2:Person)
MERGE (p)-[r:WORK_WITH]-(p2);
"""


# Create Time Tree Index
create_time_tree_index="""
CREATE INDEX FOR (y:Year) ON (y.value);
"""


# Create Years
create_years="""
WITH range(2012, 2019) AS years, range(1,12) AS months
FOREACH(year IN years |
  CREATE (y:Year {value: year}));
"""

# Connect Years Sequentially
connect_years="""
MATCH (year:Year)
WITH year
ORDER BY year.value
WITH collect(year) AS years
FOREACH(i in RANGE(0, size(years)-2) |
    FOREACH(year1 in [years[i]] |
        FOREACH(year2 in [years[i+1]] |
            MERGE (year1)-[:NEXT]->(year2))));
"""

#connect each movie in the database to a node representing the year it was released, using a CREATED_ON relationship
create_created_on_rel="""
MATCH (m:Movie)
WHERE m.release_year IS NOT NULL
WITH m, toInteger(m.release_year) AS y
MATCH (year:Year {value: y})
MERGE (m)-[:CREATED_ON]->(year)
"""

queries = [create_acted_in_rel,
           create_category,
           create_type,
           create_director,
           create_countries,
           del_unused,
           create_work_with_rel,
           create_time_tree_index,
           create_years,
           connect_years,
           create_created_on_rel]

with driver.session() as session:
    for query in queries:
        session.run(query)
        
driver.close()

In [ ]:
# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Count by category
request="""
MATCH (c:Category)<-[rel:IN_CATEGORY]-(m:Movie)
WITH c.name as category,count(*) as total
RETURN  category,total
ORDER BY total DESC
"""

with driver.session() as session:
    results = session.run(request)
    df = pd.DataFrame([dict(record) for record in results])
driver.close()
df

In [17]:
# Lets get a vector index for the embeddings
kg.query("""
  CREATE VECTOR INDEX movie_description_embeddings IF NOT EXISTS
  FOR (m:Movie) ON (m.descriptionEmbedding) 
  OPTIONS { indexConfig: {
    `vector.dimensions`: 1536,
    `vector.similarity_function`: 'cosine'
  }}"""
)

[]

In [18]:
# List all the vector indices
kg.query("""
  SHOW VECTOR INDEXES
  """
)

[{'id': 1,
  'name': 'movie_description_embeddings',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['descriptionEmbedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0}]

In [19]:
# Compute the embeddings and store them in the movie node as a property
kg.query("""
    MATCH (movie:Movie) WHERE movie.description IS NOT NULL
    WITH movie, genai.vector.encode(
        movie.description, 
        "OpenAI", 
        {
          token: $openAiApiKey
        }) AS vector
    CALL db.create.setNodeVectorProperty(movie, "descriptionEmbedding", vector)
    """, 
    params={"openAiApiKey":OPENAI_API_KEY} )

[]

In [20]:
# Let's test it out
question = "What movies are about love?"

In [23]:
# Run a query against the vector embeddings i.e perform semantic search
kg.query("""
    WITH genai.vector.encode(
        $question, 
        "OpenAI", 
        {
          token: $openAiApiKey
        }) AS question_embedding
    CALL db.index.vector.queryNodes(
        'movie_description_embeddings', 
        $top_k, 
        question_embedding
        ) YIELD node AS movie, score
    RETURN movie.title, movie.description, score
    """, 
    params={"openAiApiKey":OPENAI_API_KEY,
            "question": question,
            "top_k": 5
            })

[{'movie.title': 'Manhattan Romance',
  'movie.description': 'A filmmaker working on a documentary about love in modern Manhattan becomes personally entangled in the romantic lives of his subjects.',
  'score': 0.9291431903839111},
 {'movie.title': 'Dilwale',
  'movie.description': 'A young couple falls in love, but conflicting loyalties drive them apart. Years later, fate intervenes and hands them a second chance.',
  'score': 0.919033408164978},
 {'movie.title': 'My Dear Boy',
  'movie.description': 'Heartbroken and romantically pessimistic, a commercial director meets a young artist and teaches him life-changing lessons about love.',
  'score': 0.9179035425186157},
 {'movie.title': 'Steal a Pencil for Me',
  'movie.description': 'This moving documentary chronicles the unshakeable romance between a couple who met and fell in love while imprisoned in a Nazi concentration camp.',
  'score': 0.9178396463394165},
 {'movie.title': 'Love Rain',
  'movie.description': "A story of two couple